In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
train = pd.read_csv("/gdrive/My Drive/Data/train.csv")
test = pd.read_csv("/gdrive/My Drive/Data/test.csv")
sample = pd.read_csv("/gdrive/My Drive/Data/sample_submission.csv")

In [0]:
train['game_id'] = train['game_id'].astype(np.uint16)
train['winner'] = train['winner'].astype(np.int8)
train['player'] = train["player"].astype(str)
train['time'] = train['time'].astype(np.float32)
train1 = train[train.event_contents.notnull()][['game_id','event_contents']]

In [0]:
train2 = train[["game_id","time"]]
train2 = train2.groupby("game_id").max()

In [36]:
train3 = train[["game_id","player","event"]]
train3["count"] = 1
train3 = train3.groupby(["game_id","player","event"]).sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
train3 = train3.unstack("event").fillna(0)

In [38]:
train3 = train3.reset_index().drop("player",axis=1)
train3 = train3.set_index("game_id")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [52]:
train3.head()

count                           ...                                      
event   Ability AddToControlGroup Camera  ... Right Click Selection SetControlGroup
game_id                                   ...                                      
0          34.0               2.0  444.0  ...        35.0      50.0             3.0
0          34.0               0.0  425.0  ...        28.0      57.0             1.0
1          77.0               1.0  627.0  ...       160.0     186.0            10.0
1          67.0               0.0  858.0  ...       131.0     116.0             8.0
2          69.0               6.0  413.0  ...       160.0      90.0            14.0

[5 rows x 8 columns]

In [0]:
train4 = pd.DataFrame(columns = train3.columns,index = np.arange(0,len(train["game_id"].unique())))
for i in range(len(train["game_id"].unique())):
  data = train3[train3.index==i].reset_index(drop=True)
  train4.iloc[i,:] = list((data.iloc[0,:]-data.iloc[1,:]).reset_index(drop=True))

In [85]:
train4 = train4.reset_index()
train4 = train4.stack(level=-2).iloc[:,1:].dropna(axis=0).reset_index().iloc[:,2:]
train4["time"] = train2["time"]

0            0
1            1
2            2
3            3
4            4
         ...  
38867    38867
38868    38868
38869    38869
38870    38870
38871    38871
Name: index, Length: 38872, dtype: int64

In [160]:
train5 = train[["game_id","player","species"]]
train5["player_species"] = train5.player + train5.species
train5.drop(["player","species"],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [161]:
train5.drop_duplicates(["game_id","player_species"],inplace=True)
train5_col = train5.player_species.unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [162]:
train5["count"] = 1
train5 = train5.groupby(['game_id',"player_species"]).count().unstack("player_species")
train5.fillna(0,inplace=True)
train5 = train5.stack(level=-2).reset_index().iloc[:,2:]
train5 = pd.concat([train4,train5],axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [173]:
train6 = train[["game_id","winner"]]
train6.drop_duplicates(["game_id","winner"],inplace=True)
train_y = train6["winner"].reset_index(drop=True)
train_y

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0        1
1        1
2        0
3        0
4        0
        ..
38867    1
38868    0
38869    0
38870    1
38871    0
Name: winner, Length: 38872, dtype: int8

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_sc = scaler.fit_transform(train5)
x_train, x_val , y_train, y_val = train_test_split(train_sc,train_y,test_size=0.15,shuffle=True)

In [205]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
forest_clf = RandomForestClassifier(n_estimators=500, max_depth=20,min_samples_leaf=3)
forest_scores = cross_val_score(forest_clf, x_train, y_train, cv=5,verbose=100)
forest_scores.mean()



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  ................................................................
[CV] .................................... , score=0.601, total=  21.3s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.3s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.601, total=  21.3s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   42.6s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.595, total=  22.4s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.590, total=  21.5s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.4min remaining:    0.0s
[CV]  .........................

0.5988014088206814

In [207]:
from sklearn.metrics import accuracy_score
forest_clf.fit(x_train,y_train)
y_pred = forest_clf.predict(x_val)
accuracy_score(y_val,y_pred)

0.6009260847196021

In [0]:
test['game_id'] = test['game_id'].astype(np.uint16)
test['player'] = test["player"].astype(str)
test['time'] = test['time'].astype(np.float32)
test1 = test[test.event_contents.notnull()][['game_id','event_contents']]

In [203]:
test2 = test[["game_id","time"]]
test2 = test2.groupby("game_id").max()
test3 = test[["game_id","player","event"]]
test3["count"] = 1
test3 = test3.groupby(["game_id","player","event"]).sum()
test3 = test3.unstack("event").fillna(0)
test3 = test3.reset_index().drop("player",axis=1)
test3 = test3.set_index("game_id")
test4 = pd.DataFrame(columns = test3.columns,index = np.arange(0,len(test["game_id"].unique())))
for i in range(len(test["game_id"].unique())):
  data = test3[test3.index==i+38872].reset_index(drop=True)
  test4.iloc[i,:] = list((data.iloc[0,:]-data.iloc[1,:]).reset_index(drop=True))
test4 = test4.reset_index()
test4 = test4.stack(level=-2).iloc[:,1:].dropna(axis=0).reset_index().iloc[:,2:]
test2.reset_index(drop=True,inplace = True)
test4["time"] = test2["time"]
test4

event,Ability,AddToControlGroup,Camera,ControlGroup,GetControlGroup,Right Click,Selection,SetControlGroup,time
0,-7.0,-3.0,-235.0,0.0,-34.0,119.0,11.0,5.0,7.06
1,6.0,-1.0,54.0,4.0,-104.0,-17.0,95.0,-5.0,6.07
2,4.0,0.0,25.0,0.0,-363.0,-29.0,6.0,2.0,4.41
3,37.0,7.0,158.0,0.0,231.0,-104.0,24.0,7.0,7.58
4,-1.0,2.0,-32.0,0.0,386.0,23.0,-10.0,1.0,2.50
...,...,...,...,...,...,...,...,...,...
16782,-16.0,-13.0,-60.0,0.0,178.0,-13.0,78.0,-8.0,6.14
16783,-13.0,-2.0,172.0,0.0,167.0,-16.0,214.0,7.0,9.58
16784,-22.0,0.0,2.0,-2.0,-28.0,-145.0,15.0,1.0,6.24
16785,2.0,0.0,-45.0,0.0,-105.0,-8.0,-18.0,1.0,1.17


In [206]:
test5 = test[["game_id","player","species"]]
test5["player_species"] = test5.player + test5.species
test5.drop(["player","species"],axis=1,inplace=True)
test5.drop_duplicates(["game_id","player_species"],inplace=True)
test5_col = test5.player_species.unique()
test5["count"] = 1
test5 = test5.groupby(['game_id',"player_species"]).count().unstack("player_species")
test5.fillna(0,inplace=True)
test5 = test5.stack(level=-2).reset_index().iloc[:,2:]
test5 = pd.concat([test4,test5],axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.

In [0]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)

In [9]:
train.groupby(["game_id","player_species"]).max()["time"]

game_id  player_species                                                                                                                                                                                                    
0        0           0\n1           1\n2           0\n3           0\n4           0\n           ..\n67091771    0\n67091772    1\n67091773    0\n67091774    0\n67091775    1\nName: player, Length: 67091776, dtype: int64T    7.24
1        0           0\n1           1\n2           0\n3           0\n4           0\n           ..\n67091771    0\n67091772    1\n67091773    0\n67091774    0\n67091775    1\nName: player, Length: 67091776, dtype: int64P    9.59
         0           0\n1           1\n2           0\n3           0\n4           0\n           ..\n67091771    0\n67091772    1\n67091773    0\n67091774    0\n67091775    1\nName: player, Length: 67091776, dtype: int64T    9.59
2        0           0\n1           1\n2           0\n3           0\n4           0\n           .

In [0]:
test_sc = scaler.transform(test5)
y_test = forest_clf.predict(test_sc)

In [209]:
y_test

array([1, 1, 0, ..., 1, 0, 1], dtype=int8)

In [0]:
sample["winner"] = y_test
sample.to_csv("/gdrive/My Drive/Data/sub1.csv",index=False)